# Notebook Section 1: Exploring Battery Design Parameters

# 🔋 Exploring Battery Behavior with μₗᵢ Diagrams

This notebook lets you explore how design decisions and aging phenomena affect the internal behavior of lithium-ion batteries — using the concept of *lithium chemical potential* (μₗᵢ). Each electrode is visualized as a "tank" whose fill level corresponds to its lithiation state. 

We plot μₗᵢ vs. SoL (State of Lithiation), where:

$$
U = -\frac{\mu_{\text{Li}}}{F}
$$

This provides a powerful lens to understand cell voltage, usable SoC, and degradation mechanisms.




## Setting everything up

Importing packages

In [1]:
# Adjust path if the package is not installed
import sys
sys.path.append('.')  # Assumes root contains battery_ocv_toolbox/
# Import the package
from ocv_teaching.battery_mu_plot import BatteryMuPlot
from ocv_teaching.battery_mu_comparison import BatteryMuComparison
import pandas as pd
from scipy.interpolate import interp1d


Loading data

In [2]:
NCM811_df = pd.read_csv('../ocv_data/NMC811_half_cell_ocv.csv')
Graphite_df = pd.read_csv('../ocv_data/Graphite_half_cell_ocv.csv')
def ocv_anode(soc):
    """Returns the OCV of the anode at a given SoC."""
    f = interp1d(Graphite_df['SOC'], Graphite_df['Voltage'], bounds_error=False, fill_value="extrapolate")
    return f(soc)
def ocv_cathode(soc):
    """Returns the OCV of the cathode at a given SoC."""
    f = interp1d(NCM811_df['SOC'], NCM811_df['Voltage'], bounds_error=False, fill_value="extrapolate")
    return f(soc)


## 🎚️ SoC, Voltage Limits, and NP Ratio

Instead of defining SoC directly from capacity, we derive it from voltage limits:

- **SoC = 0** corresponds to the lower cutoff voltage (Vmin)  
- **SoC = 1** corresponds to the upper cutoff voltage (Vmax)

This lets us study how design choices shift the lithiation range within physical limits.

### ⚖️ NP Ratio (Capacity Balancing)

The **NP ratio** (Anode-to-Cathode capacity ratio) controls how much lithium the anode can absorb, relative to the cathode:

$$
\text{NP ratio} = \frac{\text{Anode capacity}}{\text{Cathode capacity}}
$$

Changing the NP ratio:
- Alters the voltage profile at a fixed SoC
- Impacts the risk of lithium plating
- Shifts how SoL is split between electrodes

Try varying NP ratio interactively to see how the curves move!

---

In [3]:
from ipywidgets import interact, FloatSlider

def interactive_mu_plot(soc=0.5, np_ratio=1.0):
    plotter = BatteryMuPlot(
        ocv_anode=ocv_anode,
        ocv_cathode=ocv_cathode,
        np_ratio=np_ratio,
        v_min=2.5,
        v_max=4.2
    )
    plotter.set_soc(soc)
    plotter.annotate_soc()

interact(
    interactive_mu_plot,
    soc=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, description="SoC"),
    np_ratio=FloatSlider(min=0.8, max=1.5, step=0.01, value=1.0, description="NP Ratio")
)


interactive(children=(FloatSlider(value=0.5, description='SoC', max=1.0, step=0.01), FloatSlider(value=1.0, de…

<function __main__.interactive_mu_plot(soc=0.5, np_ratio=1.0)>

## 🧓 Aging Modes and Their Impact

As batteries age, they degrade through several mechanisms. This tool supports three key aging modes:

- **LLI (Loss of Lithium Inventory)**  
  → Reduces total lithium available to shuttle between electrodes

- **LAM_cath (Loss of Active Material - Cathode)**  
  → Shrinks usable cathode capacity (SoL_cath range)

- **LAM_an (Loss of Active Material - Anode)**  
  → Shrinks usable anode capacity (SoL_an range)

These aging effects:
- Shift the lithium balance
- Change the Δμ between electrodes (i.e., the cell voltage)
- Reduce usable SoC even if voltage limits stay constant

---

## 🆚 BOL vs. Aged Cell Comparison

Use the comparison tool to overlay **BOL** and **aged** behavior in one diagram:

- **Dashed curves** show the BOL reference
- **Solid curves** show the aged cell
- Fill areas illustrate how lithium fills each electrode
- An arrow shows the Δμ corresponding to the full-cell voltage

This makes it easy to **see what changed**:  
Is the voltage lower at the same SoC? Has the lithium shifted to one side? Are the electrodes asymmetrically constrained?

---

## 💡 Suggested Explorations

- Set SoC = 0.5 and vary **NP ratio**. What happens to the voltage?
- Fix NP ratio and apply **LLI**. Does the anode SoL move?
- Try different combinations of **LAM_anode** and **LAM_cathode**. When do you lose SoC range?

---

This visualization gives a clear, physical sense of how batteries behave — and how design and aging change the electrochemical balance inside the cell.

In [4]:
from ipywidgets import interact, FloatSlider

def interactive_aging_comparison(soc=0.5, lli=0.0, lam_cath=0.0, lam_anode=0.0):
    # Create BOL plot
    bol = BatteryMuPlot(ocv_anode, ocv_cathode, np_ratio=1.1, v_min=2.5, v_max=4.2)
    bol.set_soc(soc)

    # Create aged plot with aging mode
    aged = BatteryMuPlot(ocv_anode, ocv_cathode, np_ratio=1.0, v_min=2.5, v_max=4.2)
    aged.set_aging_mode('custom', lli=lli, lam_cath=lam_cath, lam_anode=lam_anode)
    aged.set_soc(soc)

    # Compare and plot
    comp = BatteryMuComparison(aged, bol)
    comp.plot_comparison()
interact(
    interactive_aging_comparison,
    soc=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, description="SoC"),
    lli=FloatSlider(min=0.0, max=0.2, step=0.01, value=0.0, description="LLI"),
    lam_cath=FloatSlider(min=0.0, max=0.2, step=0.01, value=0.0, description="LAM cath"),
    lam_anode=FloatSlider(min=0.0, max=0.2, step=0.01, value=0.0, description="LAM anode"),
);


interactive(children=(FloatSlider(value=0.5, description='SoC', max=1.0, step=0.01), FloatSlider(value=0.0, de…